In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from functools import reduce
from collections import OrderedDict

## Curated dataset
read in [dataset 6](https://data.ca.gov/dataset/drinking-water-%E2%80%93-public-water-system-annually-reported-water-production-and-delivery-0) (for comparison reasons):  
Public System Water Delivered Reported in the Electronic Annual Report (EAR)*	
Monthly Water Delivery Reported by Water Suppliers in the Electronic Annual Reports (2013-2016)	 
SWRCB- State Water Resources Control Board

In [2]:
Delivery_EAR_2013_2016_df = pd.read_csv('inputData/exampleDataset6.csv') 

In [3]:
curated_columns = list(Delivery_EAR_2013_2016_df.columns)

In [4]:
new_list = []

for column_name in curated_columns:
    new_list.append(column_name.strip())
    
curated_columns = new_list

In [5]:
curated_columns

['PWSID',
 'Water.System.Name',
 'Water.System.Classification',
 'Year',
 'Month',
 'Date',
 'Days.In.Month',
 'Delivered.Water.Units AS ORIGINALLY REPORTED',
 'Delivered.Water.Units.Revised BY OFFICE OF INFORMATION MANAGEMENT AND ANALYSIS',
 'UNITS ADJUSTED BY OIMA?',
 'WATER DELIVERIES TO Single.family.Residential',
 'WATER DELIVERIES TO  Multi.family.Residential',
 'WATER DELIVERIES TO  Commercial.Institutional',
 'WATER DELIVERIES TO  Industrial',
 'WATER DELIVERIES TO  Landscape.Irrigation',
 'WATER DELIVERIES TO  Other',
 'WATER DELIVERIES TO  Agricultural',
 'WATER DELIVERIES TO  Other.PWS',
 'WATER DELIVERIES Total.Delivered Residential IN REVISED UNITS (Total Does not include Landscape Irrigation, Agricultural or to other PWS)',
 'Total. RESIDENTIAL Delivered.Gallons (Total Does not include Landscape Irrigation, Agricultural or to other PWS)',
 'Population Of Service Area',
 'CALCULATED GPCD (Total delivery to residential in gallons per capita day)']

## messy data set work

Import the raw data file into pandas

In [6]:
xls = pd.ExcelFile('inputData/EAR_Raw_2016.xlsx')

Read in each sheet of the raw data file into a dataframe. All sheets contain the Public Water System ID (PWSID) for the corresponding system. This file contains the following raw data:

Sheet 1 contains information about each individual water system and total water usage

Sheet 2 contains month-by-month water production data

Sheet 3 has CCC, Conservation, and Complaint data

Sheet 4 has Source, Rate, Delivery (month-by-month), Quality, Recycled Water, Treatment, and Emergency data for Large Water Systems (LWS)

Sheet 5 has Source, Rate, Delivery (month-by-month), Quality, and Violation data for Small Water Systems (SWS)

In [7]:
data1 = pd.read_excel(xls,'Table1_Final')
data2 = pd.read_excel(xls,'Table2_Final')
data3 = pd.read_excel(xls,'Table3_Final')
data4 = pd.read_excel(xls,'Table4_LWS')
data5 = pd.read_excel(xls,'Table5_SWS')

Concatenate the last two sheets, since each water system only appears once across the two sheets and there are some columns that only apply to large (or to small) water systems.

In [8]:
frames = [data4,data5]
data45 = pd.concat(frames, sort=True)

Merge dataframes into a single dataframe using the PWSID.

In [9]:
dfs = [data1,data2,data3,data45]
dataAll = reduce(lambda left,right: pd.merge(left,right,on='PWSID'),dfs)

Now that we have the data frame containing all of the data, set the PWSID as the index for easier analysis.

In [10]:
dataAll.set_index('PWSID',inplace=True)

In [11]:
dataAll.head().T

PWSID,CA0103040,CA0103041,CA0105002,CA0105003,CA0105008
Water System Name,NORRIS CANYON PROPERTY OWNERS ASSN,TRAILER HAVEN MOBILE HOME PARK,RIVERS END MARINA,CEMEX/ELIOT PLANT,CASTLEWOOD DOMESTIC WATER SYSTEM
Water System Classification,Community Water System,Community Water System,,,Community Water System
Water System Ownership,Privately owned Mutual Water Company or Associ...,"Privately owned, non-PUC-regulated (Community ...",Privately owned business (non-community),Privately owned business (non-community),-
Physical location Address 1,PO Box 2844,2399 E 14TH STREET SPC 28,6020 LINDEMAN RD,1544 Stanley Blvd,707 COUNTRY CLUB CIRCLE
Physical location Address 2,,,,,
Physical location City,CASTRO VALLEY,SAN LEANDRO,DISCOVERY BAY,PLEASANTON,PLEASANTON
Physical location Zip,94552,94577,94505,94566,94566
Office Phone,NaN,NaN,209-835-8365,(925)846-2824,NaN
Website URL,NaN,NaN,riversendmarina.com,NaN,www.cityofpleasantonca.gov
Reporter Name,Katherine Montgomery,Charles Keen,Sandy Mize,Rosa Maria Fibla Matamoros,Dan Martin


~~Define the columns you want in the output file. Careful: the column names defined here must match the ones used in the analysis loop! Also define dictionaries for storing month information and unit conversions.~~  
Columns previously defined by curated dataset section

In [12]:
months = {'Jan':31,'Feb':28,'Mar':31,'Apr':30,'May':31,'Jun':30,'Jul':31,'Aug':31,
          'Sep':30,'Oct':31,'Nov':30,'Dec':31}
convToGal = {'G':1,'MG':1000000,'AF':325851,'CCF':748,'HG':100,'DG':10,'CF':7.48}

Now, loop over each PWSID in the raw data and then over each month. This could in principle be expanded to loop over multiple years as well. For each month, make a dictionary using the column lists, and then fill the dictionary accordingly. Add the dictionary to a list of dictionaries, and then make a Pandas DataFrame using this list of dictionaries.

In [13]:
months.items()

dict_items([('Jan', 31), ('Feb', 28), ('Mar', 31), ('Apr', 30), ('May', 31), ('Jun', 30), ('Jul', 31), ('Aug', 31), ('Sep', 30), ('Oct', 31), ('Nov', 30), ('Dec', 31)])

In [14]:
curated_columns

['PWSID',
 'Water.System.Name',
 'Water.System.Classification',
 'Year',
 'Month',
 'Date',
 'Days.In.Month',
 'Delivered.Water.Units AS ORIGINALLY REPORTED',
 'Delivered.Water.Units.Revised BY OFFICE OF INFORMATION MANAGEMENT AND ANALYSIS',
 'UNITS ADJUSTED BY OIMA?',
 'WATER DELIVERIES TO Single.family.Residential',
 'WATER DELIVERIES TO  Multi.family.Residential',
 'WATER DELIVERIES TO  Commercial.Institutional',
 'WATER DELIVERIES TO  Industrial',
 'WATER DELIVERIES TO  Landscape.Irrigation',
 'WATER DELIVERIES TO  Other',
 'WATER DELIVERIES TO  Agricultural',
 'WATER DELIVERIES TO  Other.PWS',
 'WATER DELIVERIES Total.Delivered Residential IN REVISED UNITS (Total Does not include Landscape Irrigation, Agricultural or to other PWS)',
 'Total. RESIDENTIAL Delivered.Gallons (Total Does not include Landscape Irrigation, Agricultural or to other PWS)',
 'Population Of Service Area',
 'CALCULATED GPCD (Total delivery to residential in gallons per capita day)']

In [15]:
dList=[]
for index,row in dataAll.iterrows():
#for index,row in dataAll.head().iterrows():
    for month,num_days in months.items():
        sfStr = "WD " + month + " SF" ## single family residential
        mfStr = "WD " + month + " MF" ## multi family residential
        ciStr = "WD " + month + " CI" ## commerical institutional
        iStr = "WD " + month + " I" ## industrial
        liStr = "WD " + month + " LI" ## landscape irrigation
        oStr = "WD " + month + " O" ## other
        totStr = "WD " + month + " Total" ## full total?
        agStr = "WD " + month + " A" ## agricultural
        opwsStr = "WD " + month + " OP" ## other PWS
        
        ## if value is an empty string ' ' then replace with a 0 
        temp_keys = [sfStr,mfStr,ciStr,iStr,liStr,oStr,totStr,agStr,opwsStr]
        temp_dict = {}
        
        for i in temp_keys:
            if row[i] == ' ':
                temp_dict[i] = 0
            else:
                temp_dict[i] = row[i]

        
        dTemp = OrderedDict.fromkeys(curated_columns)
        dTemp['PWSID'] = index
        dTemp['Water.System.Name'] = row['Water System Name']
        dTemp['Water.System.Classification'] = row['Water System Classification']
        dTemp['Year'] = 2016 #This is hardcoded for now, but this is not a good permanent solution
        dTemp['Month'] = month
        dTemp['Date'] = month + " 1, 2016"
        dTemp['Days.In.Month'] = num_days
        dTemp['Delivered.Water.Units AS ORIGINALLY REPORTED'] = row['WP Units of Measure']
        #dTemp['Delivered.Water.Units.Revised BY TEAM'] = 'No'
        #dTemp['UNITS ADJUSTED BY TEAM?'] = 'No'
        dTemp['WATER DELIVERIES TO Single.family.Residential'] = temp_dict[sfStr]
        dTemp['WATER DELIVERIES TO  Multi.family.Residential'] = temp_dict[mfStr]
        dTemp['WATER DELIVERIES TO  Commercial.Institutional'] = temp_dict[ciStr]
        dTemp['WATER DELIVERIES TO  Industrial'] = temp_dict[iStr]
        dTemp['WATER DELIVERIES TO  Landscape.Irrigation'] = temp_dict[liStr]
        dTemp['WATER DELIVERIES TO  Other'] = temp_dict[oStr]
        dTemp['WATER DELIVERIES TO  Agricultural'] = temp_dict[agStr]
        dTemp['WATER DELIVERIES TO  Other.PWS'] = temp_dict[opwsStr]
        dTemp['WATER DELIVERIES Total.Delivered Residential IN ORIGINAL UNITS (Total Does not include Landscape Irrigation, Agricultural or to other PWS)'] = (
            temp_dict[sfStr] + temp_dict[mfStr] + temp_dict[ciStr] + temp_dict[iStr] + temp_dict[oStr])
        #dTemp['WATER DELIVERIES Total.Delivered Residential IN REVISED UNITS (Total Does not include Landscape Irrigation, Agricultural or to other PWS)']
                                                                                               
        num_gals = 0
        totFl = 0
        try:
            totFl = float(temp_dict[sfStr] + temp_dict[mfStr] + temp_dict[ciStr] + temp_dict[iStr] + temp_dict[oStr])
        except ValueError:
            totFl = 0
        if row['WD Unit of Measure'] in convToGal:
            convFactor = convToGal[row['WD Unit of Measure']]
            num_gals = totFl*convFactor
        dTemp['Total. RESIDENTIAL Delivered.Gallons (Total Does not include Landscape Irrigation, Agricultural or to other PWS)'] = num_gals
        try:
            num_pop = float(row['Population'])
        except ValueError:
            num_pop = 0
        dTemp['Population Of Service Area'] = num_pop
        gpcd = 0
        if (num_pop > 0 and num_days > 0): 
            gpcd = (num_gals/num_pop)/num_days
        dTemp['CALCULATED GPCD (Total delivery to residential in gallons per capita day)'] = gpcd
        dList.append(dTemp)
dfProd = pd.DataFrame(dList)
dfProd

,PWSID,Water.System.Name,Water.System.Classification,Year,Month,Date,Days.In.Month,Delivered.Water.Units AS ORIGINALLY REPORTED,Delivered.Water.Units.Revised BY OFFICE OF INFORMATION MANAGEMENT AND ANALYSIS,UNITS ADJUSTED BY OIMA?,...,WATER DELIVERIES TO Industrial,WATER DELIVERIES TO Landscape.Irrigation,WATER DELIVERIES TO Other,WATER DELIVERIES TO Agricultural,WATER DELIVERIES TO Other.PWS,"WATER DELIVERIES Total.Delivered Residential IN REVISED UNITS (Total Does not include Landscape Irrigation, Agricultural or to other PWS)","Total. RESIDENTIAL Delivered.Gallons (Total Does not include Landscape Irrigation, Agricultural or to other PWS)",Population Of Service Area,CALCULATED GPCD (Total delivery to residential in gallons per capita day),"WATER DELIVERIES Total.Delivered Residential IN ORIGINAL UNITS (Total Does not include Landscape Irrigation, Agricultural or to other PWS)"
0,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Jan,"Jan 1, 2016",31,G,None,None,...,0.0,0.00,0.0,0.0,0.0,None,1.264030e+05,50.0,81.550323,1.264030e+05
1,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Feb,"Feb 1, 2016",28,G,None,None,...,0.0,0.00,0.0,0.0,0.0,None,1.182480e+05,50.0,84.462857,1.182480e+05
2,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Mar,"Mar 1, 2016",31,G,None,None,...,0.0,0.00,0.0,0.0,0.0,None,1.393990e+05,50.0,89.934839,1.393990e+05
3,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Apr,"Apr 1, 2016",30,G,None,None,...,0.0,0.00,0.0,0.0,0.0,None,1.277100e+05,50.0,85.140000,1.277100e+05
4,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,May,"May 1, 2016",31,G,None,None,...,0.0,0.00,0.0,0.0,0.0,None,1.714400e+05,50.0,110.606452,1.714400e+05
5,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Jun,"Jun 1, 2016",30,G,None,None,...,0.0,0.00,0.0,0.0,0.0,None,2.376450e+05,50.0,158.430000,2.376450e+05
6,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Jul,"Jul 1, 2016",31,G,None,None,...,0.0,0.00,0.0,0.0,0.0,None,2.937380e+05,50.0,189.508387,2.937380e+05
7,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Aug,"Aug 1, 2016",31,G,None,None,...,0.0,0.00,0.0,0.0,0.0,None,2.767550e+05,50.0,178.551613,2.767550e+05
8,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Sep,"Sep 1, 2016",30,G,None,None,...,0.0,0.00,0.0,0.0,0.0,None,2.061230e+05,50.0,137.415333,2.061230e+05
9,CA0103040,NORRIS CANYON PROPERTY OWNERS ASSN,Community Water System,2016,Oct,"Oct 1, 2016",31,G,None,None,...,0.0,0.00,0.0,0.0,0.0,None,1.458250e+05,50.0,94.080645,1.458250e+05


In [16]:
dfProd[:1].T

,0
PWSID,CA0103040
Water.System.Name,NORRIS CANYON PROPERTY OWNERS ASSN
Water.System.Classification,Community Water System
Year,2016
Month,Jan
Date,"Jan 1, 2016"
Days.In.Month,31
Delivered.Water.Units AS ORIGINALLY REPORTED,G
Delivered.Water.Units.Revised BY OFFICE OF INFORMATION MANAGEMENT AND ANALYSIS,None
UNITS ADJUSTED BY OIMA?,None


In [17]:
dfProd.dtypes

PWSID                                                                                                                                          object
Water.System.Name                                                                                                                              object
Water.System.Classification                                                                                                                    object
Year                                                                                                                                            int64
Month                                                                                                                                          object
Date                                                                                                                                           object
Days.In.Month                                                                                       

In [18]:
## hmmmm
for i in dfProd.columns:
    if i not in curated_columns:
        print(i)

## hmmmm
for i in curated_columns:
    if i not in dfProd.columns:
        print(i)

WATER DELIVERIES Total.Delivered Residential IN ORIGINAL UNITS (Total Does not include Landscape Irrigation, Agricultural or to other PWS)


Potential quality control issues:

-Units could be incorrect. This can be addressed by calculating the GPCD and comparing to average values. 

-Numerical values (e.g. population, number of gallons) could be given as strings. 

-Missing values. Since this report is filed each year by all public water systems, we can use the redundancy in the data to your advantage. For instance, if the population of the PWS is not given in 2017, we can use the 2016 value as a reasonable estimate.

Further improvements: the analysis loop and output can be easily manipulated to produce the desired data format. More quality control can be integrated (either within or outside of the main analysis loop). 

Finally, we write the output to csv format.

In [19]:
dfProd.to_csv("outputData/WaterDelivered.csv", index=False)